<img align="right" src="images/tf-small.png" width="90"/>
<img align="right" src="images/etcbc.png" width="100"/>

# Annotating Coreference

## 1. Introduction 

This notebook contains a simple function `show_text()` that helps with annotating coreference information in the Psalms: NP's, named entities, verbs, personal pronouns (IPP's, suffixes), demonstratives and sometimes adverbs. Below I explain what information `show_text()` displays in which colours and what kind of additional ETCBC resources I use to annotate. 


## 1.1 Annotating with `show_text`
The function below `show_text()` in §2 retrieves all kinds of information from TF that is important for coreference annotation. Different coreference information is indicated with the [CSS colour codes](https://developer.mozilla.org/en-US/docs/Web/CSS/color_value): 

* Noun Phrases (`NP`): skyblue
* Appositions (`Appo`): yellow
* Verbs (`verb`): springgreen
* Substantives/Nouns (`subs`): skyblue
* Articles (`art`): skyblue
* Proper Nouns (`nmpr`): tomato
* Independent Personal Pronouns (`prps`): palegoldenrod
* Demonstrative Pronouns (`prde`): royalblue
* Suffixes that are attached to Prepositional Phrases (`PP`): DarkGoldenrod

Additional features are displayed by passing them to a string in `A.prettySetup()`: 

* The features `pgn_prps`, `pgn_prde`, `pgn_verb`, `pgn_prs` give the Person, Gender and Number (PGN) of respectively indepedent personal pronouns, demonstrative pronouns, verbs and suffixes in one 'package' of information instead of three separate features (`ps`, `gn`, `nu`). Especially for large texts these features save considerable space in `A.show()`.
* `pdp` indicates the Phrase Dependent Part of Speech;
* For feature `typ` PP or NP the determination (`det`) is important;
* For `sub`'s within PP's and NP's the status (`st`) is relevant;
* `ls` indicates the Lexical Set of words and/or lexemes. The feature contains some Named Entity information.
* `nametype` contains Named Entity information of the lexeme.
* `gn` and `nu` indicate the gender and number of the word other than the features as described under the first bullet above. 
* For ease of grammatical reference `rela` and `function` have been included. 

In the function `show_text()` in the cell below fill in the values for Hebrew Bible book and chapters that you want to have displayed with the coreference information which has been described above. 

## 1.2 Annotating with resources

Besides this function I have included in the folder **annotation_resources** resources from the ETCBC server that I use as reference works to check how certain Biblical Hebrew words are built up: 

* **alphabet** and **graphemes**: definition of the Latin transliteration alphabet of Biblical Hebrew 
* **anwb**: Analytical Dictionary for Biblical Hebrew 
* **lexicon**: contains all lexemes in Biblical Hebrew
* **named_entities**: contains categories of named entities that need to be annotated
* **word_grammar**: grammar at word level in Biblical Hebrew  

As I try to work in a [Open Science](https://en.wikipedia.org/wiki/Open_science) way, I have included the **annotation resources** here because only authorised ETCBC personnel is allowed on the ETCBC server. 

To illustrate the use of the **annotation resources**:

In Psalm 1:4 I come across the verb: 

> תִּדְּפֶ֥נּוּ (scatter), or in text-trans-plain: TDPNW (see the txt file of Psalm 1 in the NB: '1. file_preparation_for_annotation')

with PGN (Person, Gender and Number) of the verb P3Fsg and PGN of the attached suffix P3Msg. 

Say that I'm unsure how I should seperately annotate the verb and the suffix:

* Click on the word (or load the feature `lex` in `A.prettySetup()`), this leads to all occurences of that word in the website version of the BHSA: [SHEBANQ](https://shebanq.ancient-data.org/hebrew/word?version=2017&id=1NDPv). 
* Find the disambiguated verb form `NDP[` on SHEBANQ in the box on the left. The `[` indicates that the word is a verb. 
* Open the **anwb** resource in a .txt editor and look (ctrl-F, regex) for the pattern `# NDP[`. This search will bring you to all occurrences of the verb within the **anwb**. 
* The search returns: `T.ID.:PEN.W.			!T=!(NDP[~N+(HW			# NDP[`. 

    1. The first form `T.ID.:PEN.W.` is the written form in the text. It can be deciphered using the **alphabet** and **graphemes** resources. 
    2. The second form `!T=!(NDP[~N+(HW` is ETCBC's analysed form, it can be deciphered with the **word_grammar** resource. The analysed form is the most important for the annotation process. The `~N` indicates the paragogic nun; `(` a non-realised `H`. The `W` is therefore the suffix, and should be annotated. Consequently, `TDP` should be annotated as verb. 
    3. The third form `# NDP[` is the search pattern. 

## 2. Import modules and utils

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os, re, pickle, csv, collections
from IPython.display import display, HTML
from pprint import pprint

from utils import * 
from print_datetime import *

print_datetime()

Notebook last updated by Christiaan at 2019-09-28 14:09:39.598944


## 3. Import Text-Fabric Data 

For the coreference data I used the frozen 2017 version, taken from the ETCBC 2017-10-06. The 2017 version is archived in Zenodo: DOI: [doi.org/10.5281/zenodo.1007624](https://zenodo.org/record/1302798#.W5ocuC1g3pI). 

I use 2017 because I don't want the data to change while I'm annotating. The annotated data will be made available through TF later on. 

In [4]:
from tf.app import use

In [5]:
A = use(
    'bhsa', version='2017',
    mod=(
        #'etcbc/lingo/heads/tf,'
        'cmerwich/bh-reference-system/tf'
    ),
    hoist=globals(),
)

	connecting to online GitHub repo annotation/app-bhsa ... connected
Using TF-app in /Users/Christiaan/text-fabric-data/annotation/app-bhsa/code:
	rv1.2=#5fdf1778d51d938bfe80b37b415e36618e50190c (latest release)
	connecting to online GitHub repo etcbc/bhsa ... connected
Using data in /Users/Christiaan/text-fabric-data/etcbc/bhsa/tf/2017:
	rv1.6=#bac4a9f5a2bbdede96ba6caea45e762fe88f88c5 (latest release)
	connecting to online GitHub repo etcbc/phono ... connected
Using data in /Users/Christiaan/text-fabric-data/etcbc/phono/tf/2017:
	r1.2 (latest release)
	connecting to online GitHub repo etcbc/parallels ... connected
Using data in /Users/Christiaan/text-fabric-data/etcbc/parallels/tf/2017:
	r1.2 (latest release)
	connecting to online GitHub repo cmerwich/bh-reference-system ... connected
Using data in /Users/Christiaan/text-fabric-data/cmerwich/bh-reference-system/tf/2017:
	rv1.0 (latest release)


## 4. TF query for appositions

In coreference appositional relations like: "Barack Obama, the President of the United States, is coming to visit the Rijksmuseum." (He did actually.) In the example "the President of the United States" is an apposition to Barack Obama. 

The TF-query looks for such appositions in the Psalms. The results can be used for reference during the annotation process. The `show_text` function below highlights the apposition in phrase atoms in yellow.

With `A.show(res, start=1, end=30)` in the second cell the results can be visualised in context of the text with all kinds of features. Tweak `end=30` to iniducate how many results you want to see.

### 4.1 Run TF-query

In [ ]:
res = A.search('''
book book=Psalmi
  verse
    phrase_atom rela=Appo
''')

A.table(res, linked=3)

### 4.2 Appositions in context  

In [ ]:
A.show(res, start=1, end=30)

## 5 Annotation aid: `show_text()`  
- Run the cells below first - 

In [6]:
def compute_text(MY_BOOK, MY_CHAPTERS):

    results = []
    highlights = {}
    
    for book in F.otype.s('book'):
        book_name = T.bookName(book)
       
        for chn in L.d(book, 'chapter'):
            chapter = F.chapter.v(chn)
            tup = (chn,)
            if (
                (MY_BOOK and book_name not in MY_BOOK)
                or 
                (MY_CHAPTERS and chapter not in MY_CHAPTERS)
            ):
                continue
            for phrase in L.d(chn, 'phrase'):
                typ = F.typ.v(phrase)
                if typ == 'NP':
                    tup = tup + (phrase,)
                    highlights[phrase] = 'skyblue'

            for phr_atom in L.d(chn, 'phrase_atom'):
                if F.rela.v(phr_atom) == 'Appo':
                    tup = tup + (phr_atom,)
                    highlights[phr_atom] = 'yellow'

            for w in L.d(chn, 'word'):
                pdp = F.pdp.v(w)
                pgn_prps = F.pgn_prps.v(w)
                pgn_prde = F.pgn_prde.v(w)
                pgn_verb = F.pgn_verb.v(w)
                pgn_prs = F.pgn_prs.v(w)

                if pdp == 'verb':
                    tup = tup + (w,)
                    highlights[w] = 'springgreen'

                if pdp == 'subs':
                    tup = tup + (w,)
                    highlights[w] = 'skyblue'

                if pdp == 'art':
                    tup = tup + (w,)
                    highlights[w] = 'skyblue'

                if pdp == 'nmpr':
                    tup = tup + (w,)
                    highlights[w] = 'tomato' 

                if pdp == 'prps':
                    tup = tup + (w,)
                    highlights[w] = 'palegoldenrod'

                if pdp == 'prde':
                    tup = tup + (w,)
                    highlights[w] = 'royalblue'

                if pdp == 'prep' and pgn_prs in prs_set:
                    tup = tup + (w,)
                    highlights[w] = 'DarkGoldenrod'

            results.append(tup)
    return (results, highlights)

In [7]:
def show_text(results, highlights):
    A.displaySetup(withNodes=True, extraFeatures='pgn_prps pgn_prde pgn_verb pgn_prs pdp typ rela ls function det st lex nametype gn nu') #ps gn nu
    A.show(results, condensed=False, highlights=highlights)

### 5.1 Choose book and chapter

In [8]:
# Set any Hebrew Bible Book
MY_BOOK = {'Numbers'}

# Indicate a range of chapters
MY_CHAPTERS = set(range(10,11)) 

# Run
(results, highlights) = compute_text(MY_BOOK, MY_CHAPTERS)

### `show_text()`

Run the cell below each time you choose a new book and/or chapter. 

In [9]:
show_text(results, highlights)